In [5]:
data = []
with open('relations.csv') as f:
    data = [row[:-1].split(',') for row in f][1:]

In [6]:
def process_r(r):
    r = r.replace('-', '_').replace('/', '_or_').replace(' ', '_')
    char_lst=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    
    res = ""
    for i in r:
        if i == '_':
            res += i
        else:
            if i in char_lst:
                res += i
                
    return res.upper()

In [7]:
with open('visulization.cypher', 'w') as f:
    f.write('LOAD CSV WITH HEADERS FROM "file:///nodes.csv" AS line\n')
    f.write('MERGE (p:person{id:line.id, name:line.name, blood_status:line.blood_status, hair_color:line.hair_color, eye_color:line.eye_color, patronus:line.patronus, wand:line.wand, occupation:line.occupation, loyalty:line.loyalty, house:line.house});\n\n')

    for row in data:
        f.write("MATCH (from:person{id:'"+row[0]+"'}),(to:person{id:'"+row[2]+"'})\n")
        f.write("MERGE (from)-[r:{}]->(to);\n".format(process_r(row[1])))

In [8]:
data

[['0', 'enemy', '217'],
 ['0', 'enemy', '197'],
 ['0', 'enemy', '226'],
 ['0', 'enemy', '243'],
 ['0', 'enemy', '244'],
 ['0', 'enemy', '245'],
 ['0', 'enemy', '246'],
 ['0', 'enemy', '247'],
 ['0', 'enemy', '248'],
 ['0', 'enemy', '249'],
 ['0', 'enemy', '250'],
 ['0', 'enemy', '251'],
 ['0', 'enemy', '32'],
 ['0', 'enemy', '164'],
 ['0', 'enemy', '165'],
 ['0', 'enemy', '255'],
 ['0', 'enemy', '256'],
 ['0', 'enemy', '257'],
 ['0', 'enemy', '258'],
 ['0', 'enemy', '192'],
 ['0', 'enemy', '260'],
 ['0', 'enemy', '261'],
 ['0', 'enemy', '188'],
 ['0', 'enemy', '263'],
 ['0', 'enemy', '264'],
 ['0', 'enemy', '265'],
 ['0', 'enemy', '179'],
 ['0', 'enemy', '175'],
 ['0', 'enemy', '268'],
 ['0', 'enemy', '269'],
 ['0', 'enemy', '270'],
 ['0', 'enemy', '201'],
 ['0', 'enemy', '196'],
 ['0', 'enemy', '273'],
 ['0', 'enemy', '274'],
 ['0', 'enemy', '216'],
 ['0', 'enemy', '237'],
 ['1', 'daughter', '59'],
 ['1', 'father-in-law', '24'],
 ['1', 'nephew/godson', '6'],
 ['1', 'nephew', '182'],
 